In [1]:
import os
import numpy as np
import cv2
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

In [2]:
# تابع استخراج ویژگی
# -----------------------
def extract_histogram_features_fixed_resize(image_path, resize_shape=(200, 200), gray_levels=8, grid_size=(10, 10)):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, resize_shape)

    patch_h = resize_shape[0] // grid_size[0]
    patch_w = resize_shape[1] // grid_size[1]

    features = []

    grad_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = cv2.magnitude(grad_x, grad_y)

    gray_bin_size = 256 // gray_levels
    grad_max = gradient_magnitude.max()
    grad_bin_size = grad_max / gray_levels if grad_max != 0 else 1

    for i in range(grid_size[0]):
        for j in range(grid_size[1]):
            y1, y2 = i * patch_h, (i + 1) * patch_h
            x1, x2 = j * patch_w, (j + 1) * patch_w

            patch = img[y1:y2, x1:x2]
            grad_patch = gradient_magnitude[y1:y2, x1:x2]

            hist_intensity, _ = np.histogram(patch, bins=gray_levels, range=(0, 256))
            hist_intensity = hist_intensity.astype(np.float32) / (patch.size + 1e-6)

            hist_grad, _ = np.histogram(grad_patch, bins=gray_levels, range=(0, grad_max))
            hist_grad = hist_grad.astype(np.float32) / (grad_patch.size + 1e-6)

            patch_features = np.concatenate([hist_intensity, hist_grad])
            features.append(patch_features)

    return np.concatenate(features)

In [3]:
# لود کردن کل دیتاست
# -----------------------
def load_dataset(dataset_path, labels_dict):
    X, y = [], []
    for label_name, label_index in labels_dict.items():
        folder_path = os.path.join(dataset_path, label_name)
        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.jpg', '.png', '.jpeg')):
                image_path = os.path.join(folder_path, filename)
                features = extract_histogram_features_fixed_resize(image_path)
                X.append(features)
                y.append(label_index)
    return np.array(X), np.array(y)


In [4]:
# مسیر و لیبل‌ها
# -----------------------
dataset_path = 'processed_images/'  # مسیر دیتاست کراپ‌شده
labels_dict = {'Tumor': 0, 'Stone': 1, 'Normal': 2, 'Cyst': 3}

In [5]:
# آماده‌سازی داده‌ها
# -----------------------
X, y = load_dataset(dataset_path, labels_dict)

In [6]:

# نرمال‌سازی
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# reshape برای LSTM -> (تعداد نمونه، 100 پنجره، 16 ویژگی)
X_seq = X_scaled.reshape((X_scaled.shape[0], 100, 16))

# تبدیل لیبل به one-hot
y_cat = to_categorical(y, num_classes=4)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_seq, y_cat, test_size=0.2, random_state=42, stratify=y
)


In [11]:
# تعریف مدل LSTM
# -----------------------
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()
model.add(LSTM(200, input_shape=(100, 16)))  
model.add(Dense(4, activation='softmax'))

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(
    monitor='val_loss',      
    patience=5,               
    restore_best_weights=True
)
# آموزش مدل
# -----------------------
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])


Epoch 1/100
312/312 ━━━━━━━━━━━━━━━━━━━━ 30s 89ms/step - accuracy: 0.6339 - loss: 0.9355 - val_accuracy: 0.7478 - val_loss: 0.6864
Epoch 2/100
312/312 ━━━━━━━━━━━━━━━━━━━━ 27s 88ms/step - accuracy: 0.7386 - loss: 0.6683 - val_accuracy: 0.7602 - val_loss: 0.6294
Epoch 3/100
312/312 ━━━━━━━━━━━━━━━━━━━━ 29s 94ms/step - accuracy: 0.7562 - loss: 0.6223 - val_accuracy: 0.8297 - val_loss: 0.5043
Epoch 4/100
312/312 ━━━━━━━━━━━━━━━━━━━━ 30s 96ms/step - accuracy: 0.8110 - loss: 0.4884 - val_accuracy: 0.8727 - val_loss: 0.3423
Epoch 5/100
312/312 ━━━━━━━━━━━━━━━━━━━━ 26s 85ms/step - accuracy: 0.8582 - loss: 0.3739 - val_accuracy: 0.8530 - val_loss: 0.3949
Epoch 6/100
312/312 ━━━━━━━━━━━━━━━━━━━━ 26s 83ms/step - accuracy: 0.8886 - loss: 0.3070 - val_accuracy: 0.8956 - val_loss: 0.3072
Epoch 7/100
312/312 ━━━━━━━━━━━━━━━━━━━━ 27s 86ms/step - accuracy: 0.8890 - loss: 0.3030 - val_accuracy: 0.9000 - val_loss: 0.2713
Epoch 8/100
312/312 ━━━━━━━━━━━━━━━━━━━━ 27s 85ms/step - accuracy: 0.9209 - loss: 0

In [12]:
# ارزیابی مدل
# -----------------------
# پیش‌بینی
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

# گزارش عملکرد
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_true, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("Report:\n", classification_report(y_true, y_pred, target_names=labels_dict.keys()))


78/78 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step
Accuracy: 0.9943775100401606
Confusion Matrix:
 [[ 457    0    0    0]
 [   0  266    1    8]
 [   3    2 1011    0]
 [   0    0    0  742]]
Report:
               precision    recall  f1-score   support

       Tumor       0.99      1.00      1.00       457
       Stone       0.99      0.97      0.98       275
      Normal       1.00      1.00      1.00      1016
        Cyst       0.99      1.00      0.99       742

    accuracy                           0.99      2490
   macro avg       0.99      0.99      0.99      2490
weighted avg       0.99      0.99      0.99      2490



In [13]:
model.save('kidney_ultrasound_lstm_model.keras')
